# Import Library

In [10]:
from twython import Twython
import sqlite3
from time import sleep
import numpy as np
import pandas as pd
import pickle
import warnings
from util import JSONParser
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import string
warnings.simplefilter(action='ignore', category=FutureWarning)

# Access API

In [77]:
import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler("CkhGa0z1CKMlK7GY56Zxh0250", 
    "Iq1o1IwRzuFw6xCandQbGUX1gzx6GezPD7ek1ptbcw46eni3LB")
auth.set_access_token("492948056-n4H0mlG3efeuJnDKkkoyeW9BWbl7kBrI95Dmlup1", 
    "jMSNiTz1mo764Xnz1x8cEAuu2CQRIhRbstAgRiD5WY7cP")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


# import json parser

In [3]:
jp=JSONParser()
jp.parse("data/intents.json")

# Important Function

## open model pkl

In [ ]:
def open_model(model_path):
    with open (model_path, 'rb') as f:
        model = pickle.load(f)
    return model
model=open_model("model/model_2.pkl")

## Document Preprocessing

In [7]:
def document_processing(document):
    document = document.lower()
    document = document.translate(str.maketrans("","",string.punctuation))
    document = re.sub("[^A-Za-z\s']"," ", document)
    document = re.sub(r'http\S+', '', document) # remove links
    document = re.sub(r"www.\S+", " ", document) # remove link
    document = document.strip()
    stemmer = StemmerFactory().create_stemmer()
    document = stemmer.stem(document)
    return document

## get response

In [91]:
def response(data):
    data['text']=data['text'].apply(document_processing)
    response=[]
    for d in data['text'].values:
        proba=model.predict_proba([d])[0]
        if proba.max()<0.25: response.append("Maaf Kak Saya Tidak Mngerti")
        else:
            r=model.predict([d])[0]
            response.append(jp.get_response(r))
    return response


In [82]:
df=pd.DataFrame(data=[['atm rusak min','001002003']], columns=['text', 'sender'])

In [92]:
response(df)

['Oke kak jangan khawatir, untuk ATM rusak, ATM tertelan, dan ATM hilang akan kami ganti dengan kartu yang baru serta Gratis! Oiya jangan lupa isi data diri dulu ya di link ini https:/www.banktoyib.com/data_diri']

## connection database

In [17]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

## grab massage

In [78]:
def get_massage(api):
    pesan=api.get_direct_messages()
    df= pd.DataFrame([[int(s.id), s.created_timestamp, s.message_create['message_data']['text'], s.message_create['target']['recipient_id'], s.message_create['sender_id']] for s in pesan], columns=('ID', 'created at', 'text', 'recipient', 'sender' ))
    return df[df.sender!='492948056']

## check new massage

In [79]:
def cek_response(api, cursor):
    max_id=cursor.execute('SELECT MAX(ID) FROM direct_massage').fetchall()[0][0]
    if max_id==None:max_id=0
    new_massage=get_massage(api)
    if new_massage.ID.max()>max_id:
        massage=new_massage[new_massage.ID>max_id]
        massage['response']=response(massage['text']['user'])
        print(f"Ada {massage.shape[0]} pesan baru")
        for mas in massage.ID:
            try:
                api.send_direct_message(massage[massage.ID==mas].sender[0], massage[massage.ID==mas].response[0])
            except Exception as error:
                print(error)
        return True, massage
    else:
        return False, None
        # print("tidak ada pesan baru")

# Main Menu

## Connection Database

In [80]:
con = create_connection("twit copy.db")
cursor=con.cursor()

## run bot

In [ ]:
while True:    
    got, data=cek_response(api, cursor)
    if got: 
        data.to_sql('direct_massage', con=con, method='multi', if_exists='append', index=False)
        print(f'Data baru sejumlah {data.shape[0]}')
    else: print("No Massage")
    sleep(60)